# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("WeatherPy\weather_df.csv")
weather_df = pd.read_csv(csv_file)

# Display sample data
weather_df.head()

,Unnamed: 0,city,Temperature,Longitude,Latitude,Cloudiness,Wind Speed,Humidity
0,0,chibougamau,16.79,-74.3659,49.9168,75,1.03,82
1,1,amuntai,28.39,115.2494,-2.4177,100,1.29,68
2,2,puerto natales,-1.19,-72.4875,-51.7236,100,1.09,86
3,3,klyuchi,29.68,79.1667,52.2667,80,4.00,45
4,4,port mathurin,23.20,63.4167,-19.6833,23,10.29,74


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot_1
map_plot_1 = weather_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "city"
)

# Display the map plot_1
map_plot_1

c:\Users\User\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (city,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = weather_df.loc[(weather_df["Temperature"] >21)]
ideal_weather = ideal_weather.loc[(weather_df["Wind Speed"] < 15)]
ideal_weather = ideal_weather.loc[(weather_df["Cloudiness"] < 10)]


ideal_weather = ideal_weather.dropna()

ideal_weather

,Unnamed: 0,city,Temperature,Longitude,Latitude,Cloudiness,Wind Speed,Humidity
9,9,snyder,32.14,-100.9176,32.7179,0,3.09,29
10,10,zhezqazghan,23.32,67.7667,47.7833,0,7.00,46
24,24,fort bragg,28.01,-79.0060,35.1390,0,4.63,77
27,27,bouarfa,23.15,-1.9500,32.5300,5,2.00,19
30,30,porto velho,27.03,-63.9039,-8.7619,0,0.00,78
...,...,...,...,...,...,...,...,...
501,501,kearney,29.99,-99.0815,40.6995,0,5.66,48
507,507,crane,30.50,-102.3501,31.3974,0,5.08,28
511,511,robore,23.93,-59.7500,-18.3333,2,1.76,55
512,512,puerto quijarro,23.67,-57.7667,-17.7833,2,1.17,53


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,Unnamed: 0,city,Temperature,Longitude,Latitude,Cloudiness,Wind Speed,Humidity,Hotel Name
9,9,snyder,32.14,-100.9176,32.7179,0,3.09,29,
10,10,zhezqazghan,23.32,67.7667,47.7833,0,7.00,46,
24,24,fort bragg,28.01,-79.0060,35.1390,0,4.63,77,
27,27,bouarfa,23.15,-1.9500,32.5300,5,2.00,19,
30,30,porto velho,27.03,-63.9039,-8.7619,0,0.00,78,
...,...,...,...,...,...,...,...,...,...
501,501,kearney,29.99,-99.0815,40.6995,0,5.66,48,
507,507,crane,30.50,-102.3501,31.3974,0,5.08,28,
511,511,robore,23.93,-59.7500,-18.3333,2,1.76,55,
512,512,puerto quijarro,23.67,-57.7667,-17.7833,2,1.17,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
categories = "accommodation.hotels"
radius = 10,000 #Metres as provided
limit = 20

params = {
   "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Search for Hotels begun")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'city']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Search for Hotels begun
snyder - nearest hotel: No hotel found
zhezqazghan - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
bouarfa - nearest hotel: No hotel found
porto velho - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
corumba - nearest hotel: No hotel found
dara - nearest hotel: No hotel found
chifeng - nearest hotel: No hotel found
ypsonas - nearest hotel: No hotel found
corinto - nearest hotel: No hotel found
detroit lakes - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
hafar al-batin - nearest hotel: No hotel found
dhromolaxia - nearest hotel: No hotel found
siwa oasis - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
ilha de mocambique - nearest hotel: No hotel found
mournies - nearest hotel

,Unnamed: 0,city,Temperature,Longitude,Latitude,Cloudiness,Wind Speed,Humidity,Hotel Name
9,9,snyder,32.14,-100.9176,32.7179,0,3.09,29,No hotel found
10,10,zhezqazghan,23.32,67.7667,47.7833,0,7.00,46,No hotel found
24,24,fort bragg,28.01,-79.0060,35.1390,0,4.63,77,No hotel found
27,27,bouarfa,23.15,-1.9500,32.5300,5,2.00,19,No hotel found
30,30,porto velho,27.03,-63.9039,-8.7619,0,0.00,78,No hotel found
...,...,...,...,...,...,...,...,...,...
501,501,kearney,29.99,-99.0815,40.6995,0,5.66,48,No hotel found
507,507,crane,30.50,-102.3501,31.3974,0,5.08,28,No hotel found
511,511,robore,23.93,-59.7500,-18.3333,2,1.76,55,No hotel found
512,512,puerto quijarro,23.67,-57.7667,-17.7833,2,1.17,53,No hotel found


In [9]:
hotel_df["Hotel Name"].unique()

array(['No hotel found'], dtype=object)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "city",
    hover_cols = ["Hotel Name", "Country"],
)

# Display the map plot_1
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (city,Humidity,Hotel Name)